<a href="https://colab.research.google.com/github/amkusmec/CERIS_PAG/blob/main/CERIS_PAG_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Run the following three code blocks to load the data, packages, and functions required for CERIS-JGRA.

In [1]:
system("git clone https://github.com/amkusmec/CERIS_PAG/")


In [2]:
 if (!require(colorspace)) install.packages("colorspace")
 if (!require(rrBLUP)) install.packages("rrBLUP")

Loading required package: colorspace

Loading required package: rrBLUP

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rrBLUP’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
cwd <- '/content/CERIS_PAG/' # This is the location of the files cloned into your Google Drive
source(paste0(cwd, 'Sub_functions_PAG.r'))

# Temporary code for loading modified functions
r_files <- list.files(paste0(cwd, "R"), "*", full.names = TRUE)
for (f in r_files) source(f)

# Experiment selection

Modify the values in `""` in the next code block to select an organism and trait for analysis.

Organism | Trait Code | Trait name
-------- | ---------- | ----------
Sorghum  | FTgdd      | GDD to flowering
Wheat    | GY         | Grain yield

In [7]:
experiment <- "Sorghum"
trait <- "FTgdd"

In [5]:
exp_dir <- paste(cwd, experiment, '/', sep = '')

# Load the environment metadata file
# Column `PlantingDate` should be formatted "YYYY-MM-DD" for proper parsing
env_meta_file <- paste(exp_dir, 'Env_meta_table.txt', sep = '')
env_meta_info_0 <- read.table(env_meta_file, header = T, sep = "\t", stringsAsFactors = F)

# Load the phenotypic data
exp_traits_file <- paste(exp_dir, 'Traits_record.txt', sep = '')
exp_traits <- read.table(exp_traits_file, sep = "\t", header = T, stringsAsFactors = F, na.string = 'NA')

In [8]:
# Load the daily environmental data
all_env_codes <- unique(exp_traits$env_code)
env_cols <- rainbow_hcl(length(all_env_codes), c = 80, l = 60, start = 0, end = 300, fixup = TRUE, alpha = 0.75)

envParas_file <- paste0(exp_dir, length(all_env_codes), 'Envs_envParas.txt')
if ( !file.exists(envParas_file) ) {
  envParas <- Compile_Envirome_Matrix(exp_dir, all_env_codes, envParas_file)
} else {
  envParas <- read.table(envParas_file, sep = "\t", header = T, stringsAsFactors = F, na.string = "NA")
  if (!("DAP" %in% names(envParas))) {
    params <- setdiff(names(envParas), c("env_code", "date"))
    envParas <- split(envParas, envParas$env_code)
    envParas <- lapply(envParas, function(df) {
            df$DAP <- 1:nrow(df)
            df[, c("env_code", "DAP", params)]
    })
    envParas <- do.call("rbind", envParas)
  }
}

# Names of the environmental parameters
Paras <- names(envParas)[-(1:2)]

In [ ]:
 #### Not sure what the point of this is
 #if (experiment == '1Sorghum') { searching_days <- 122; trait <- 'FTgdd'};
 #if (experiment == '2Idaho') { searching_days <- 150; trait <- 'GY'};

In [ ]:
# Standardize the format of the trait data
lInd <- which(colnames(exp_traits) == 'line_code')
eInd <- which(colnames(exp_traits) == 'env_code')
tInd <- which(colnames(exp_traits) == trait)

exp_trait <- exp_traits[, c(lInd, eInd, tInd)]
colnames(exp_trait)[3] <- 'Yobs'

# Average across replicates within environments to reduce to one observation
# per line per environment
exp_trait <- aggregate(Yobs ~ line_code + env_code, exp_trait, mean, na.rm = TRUE)
exp_trait <- exp_trait[!is.na(exp_trait$Yobs), ]

line_codes <- unique(exp_trait$line_code)

# Calculate the environmental mean phenotype
env_mean_trait_0 <- aggregate(Yobs ~ env_code, exp_trait, mean, na.rm = TRUE)
colnames(env_mean_trait_0)[2] <- 'meanY'
env_mean_trait <- merge(env_mean_trait_0, env_meta_info_0)
env_mean_trait <- env_mean_trait[order(env_mean_trait$meanY), ]

# Finlay-Wilkinson regression

# Critical environmental regressors through informed search (CERIS)

New section